In [ ]:
!dir /b /w .

In [ ]:
import sys
import torch
sys.path.append('../python')

from MoeNet_lite2 import Net

In [ ]:
cuda = torch.cuda.is_available()
dtype = torch.float32
model = Net(4)
model.eval()
for param in model.parameters():
    param.requires_grad_(False)
weights = model.state_dict()
for name in weights.keys():
    print(name)

In [ ]:
modelPath = './model_epoch_43.pth'
o = torch.load(modelPath, map_location='cpu')
for key in o:
    print(key)

In [ ]:
w = o['model'].state_dict()
for name in w.keys():
    print(name)

In [ ]:
for name in weights.keys():
    weights[name] = w['module.' + name]

In [ ]:
def changeName(old, new):
    weights[new] = weights[old]
    del weights[old]

names = ['conv_1.weight', 'conv_2.weight', 'relu.weight', 'scale.scale']
for i in range(6):
    for name in names:
        changeName('convt_F1{}.{}'.format(i + 1, name), 'convt_F{}.{}'.format(i + 1, name))
changeName('convt_shape1.weight', 'u.weight')

In [ ]:
model.load_state_dict(weights)
torch.save(model.state_dict(), '../model/lite/model_4.pth')